<h3>06. 미니 배치와 데이터 로드</h3>

<h4>1. 미니 배치와 배치 크기</h4>
  
만약, 데이터가 수십만개 이상이라면 전체 데이터에 대해서 경사 하강법을 수행하는 것은 매우 느릴 뿐만 아니라 많은 계산량이 필요합니다. 정말 어쩌면 메모리의 한계로 계산이 불가능한 경우도 있을 수 있습니다.

그렇기 때문에 전체 데이터를 더 작은 단위로 나누어서 해당 단위로 학습하는 개념이 나오게 되었습니다.
이 단위를 미니 배치(Mini Batch)라고 합니다.

미니 배치 학습을 하게되면 미니 배치만큼만 가져가서 미니 배치에 대한 대한 비용(cost)를 계산하고, 경사 하강법을 수행합니다. 그리고 다음 미니 배치를 가져가서 경사 하강법을 수행하고 마지막 미니 배치까지 이를 반복합니다. 이렇게 전체 데이터에 대한 학습이 1회 끝나면 1 에포크(Epoch)가 끝나게 됩니다.
  
- 에포크는 전체 훈련 데이터가 학습에 한 번 사용된 주기를 말함
  
미니 배치 학습에서는 미니 배치의 개수만큼 경사 하강법을 수행해야 전체 데이터가 한 번 전부 사용되어 1 에포크(Epoch)가 됩니다. 미니 배치의 개수는 결국 미니 배치의 크기를 몇으로 하느냐에 따라서 달라지는데 미니 배치의 크기를 배치 크기(batch size)라고 합니다.
  
- 전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법을 '배치 경사 하강법' 이라 함. 반면, 미니 배치 단위로 경사 하강법을 수행하는 방법을 '미니 배치 경사 하강법'이라고 부릅니다.

- 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터를 사용하므로 가중치 값이 최적값에 수렴하는 과정이 매우 안정적이지만, 계산량이 너무 많이 듭니다. 미니 배치 경사 하강법은 경사 하강법을 할 때, 전체 데이터의 일부만을 보고 수행하므로 최적값으로 수렴하는 과정에서 값이 조금 헤매기도 하지만 훈련 속도가 빠릅니다.

- 배치 크기는 보통 2의 제곱수를 사용합니다. ex) 2, 4, 8, 16, 32, 64... 그 이유는 CPU와 GPU의 메모리가 2의 배수이므로 배치크기가 2의 제곱수일 경우에 데이터 송수신의 효율을 높일 수 있다고 합니다.

<h4>2. 이터레이션 (iteration)</h4>
  
이터레이션은 한 번의 에포크 내에서 이루어지는 매개변수인 가중치 와 의 업데이트 횟수입니다. 전체 데이터가 2,000일 때 배치 크기를 200으로 한다면 이터레이션의 수는 총 10개입니다. 이는 한 번의 에포크 당 매개변수 업데이트가 10번 이루어짐을 의미합니다.

1 epoch = batch size * iteration

<h4>3. 데이터 로드</h4>
  
파이토치에서는 데이터를 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 데이터셋(Dataset)과 데이터로더(DataLoader)를 제공합니다. 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있습니다. 기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것입니다.



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
#TensorDataset은 기본적으로 텐서를 입력으로 받는다.

In [4]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [5]:
#이제 이를 TensorDataset의 입력으로 사용하고 dataset으로 저장
dataset = TensorDataset(x_train, y_train)

파이토치의 데이터셋을 만들었다면 데이터로더를 사용 가능합니다. 데이터로더는 기본적으로 2개의 인자를 입력받는다. 하나는 데이터셋, 미니 배치의 크기입니다. 이때 미니 배치의 크기는 통상적으로 2의 배수를 사용합니다. (ex) 64, 128, 256...) 그리고 추가적으로 많이 사용되는 인자로 shuffle이 있습니다. shuffle=True를 선택하면 Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꿉니다.

사람도 같은 문제지를 계속 풀면 어느 순간 문제의 순서에 익숙해질 수 있습니다. 예를 들어 어떤 문제지의 12번 문제를 풀면서, '13번 문제가 뭔지는 기억은 안 나지만 어제 풀었던 기억으로 정답은 5번이었던 것 같은데' 하면서 문제 자체보단 순서에 익숙해질 수 있다는 것입니다. 그럴 때 문제지를 풀 때마다 문제 순서를 랜덤으로 바꾸면 도움이 될 겁니다. 마찬가지로 모델이 데이터셋의 순서에 익숙해지는 것을 방지하여 학습할 때는 이 옵션을 True를 주는 것을 권장합니다

In [6]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [7]:
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(),lr=1e-5)

In [9]:
nb_epochs =20

for epoch in range(nb_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        x_train,y_train=samples
        prediction = model(x_train)

        cost = F.mse_loss(prediction,y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader),
            cost.item()
            ))

Epoch    0/20 Batch 1/3 Cost: 12161.132812
Epoch    0/20 Batch 2/3 Cost: 2008.814941
Epoch    0/20 Batch 3/3 Cost: 537.677490
Epoch    1/20 Batch 1/3 Cost: 279.034485
Epoch    1/20 Batch 2/3 Cost: 166.758240
Epoch    1/20 Batch 3/3 Cost: 52.902451
Epoch    2/20 Batch 1/3 Cost: 7.006854
Epoch    2/20 Batch 2/3 Cost: 3.734636
Epoch    2/20 Batch 3/3 Cost: 2.889315
Epoch    3/20 Batch 1/3 Cost: 0.973634
Epoch    3/20 Batch 2/3 Cost: 0.303789
Epoch    3/20 Batch 3/3 Cost: 1.537387
Epoch    4/20 Batch 1/3 Cost: 0.281393
Epoch    4/20 Batch 2/3 Cost: 1.485680
Epoch    4/20 Batch 3/3 Cost: 1.480929
Epoch    5/20 Batch 1/3 Cost: 1.473359
Epoch    5/20 Batch 2/3 Cost: 0.639442
Epoch    5/20 Batch 3/3 Cost: 1.083872
Epoch    6/20 Batch 1/3 Cost: 0.849908
Epoch    6/20 Batch 2/3 Cost: 1.547133
Epoch    6/20 Batch 3/3 Cost: 0.050965
Epoch    7/20 Batch 1/3 Cost: 0.552800
Epoch    7/20 Batch 2/3 Cost: 1.277856
Epoch    7/20 Batch 3/3 Cost: 0.405936
Epoch    8/20 Batch 1/3 Cost: 0.661501
Epoch    8/

In [10]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 


훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[151.8669]], grad_fn=<AddmmBackward0>)
